# Introduction

While analysing the  24-26 august attack on frotly I accidentally discovered lots of files ending in *.crypt and soon discovered that frotly had an crypto virus already 18.august. This notebook will look at the crypt virus incident


# Notebook setup

In [2]:
import json
import pandas as pd
import numpy as np
import urllib.parse


from IPython.display import display, Markdown,HTML, Latex
from intake_splunk.core import SplunkConnect
from treelib import Tree,Node

from proctree import ProcTree


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 0)

s=SplunkConnect("https://localhost:8089")
s.auth("admin","superadmin")
SPL=s.read_pandas

#workaround for bug in IPython.display.JSON
def JSON(text):
    text_as_json=json.loads(text)
    print(json.dumps(text_as_json, indent=4))


Someone is encrypting the shared folders at frothly. 

In [3]:
df=SPL("""index=botsv2 *.crypt sourcetype="XmlWinEventLog:Microsoft-Windows-Sysmon/Operational" | sort _time | table _time, file_path""")
df[:15]

,_time,file_path
0,2017-08-18T21:52:49.000+00:00,C:\FROTHLY\.DS_Store.crypt
1,2017-08-18T21:52:50.000+00:00,C:\FROTHLY\Home.crypt
2,2017-08-18T21:52:51.000+00:00,C:\FROTHLY\Home\.DS_Store.crypt
3,2017-08-18T21:52:53.000+00:00,C:\FROTHLY\Home\Administrator.crypt
4,2017-08-18T21:52:54.000+00:00,C:\FROTHLY\Home\Administrator\inputs.conf.crypt
5,2017-08-18T21:52:55.000+00:00,C:\FROTHLY\Home\al.bungstien.crypt
6,2017-08-18T21:52:56.000+00:00,C:\FROTHLY\Home\amber.turing.crypt
7,2017-08-18T21:52:57.000+00:00,C:\FROTHLY\Home\amber.turing\yeast_recipes.crypt
8,2017-08-18T21:53:02.000+00:00,C:\FROTHLY\Home\amber.turing\yeast_recipes\31564-pdf.pdf.crypt
9,2017-08-18T21:53:04.000+00:00,C:\FROTHLY\Home\amber.turing\yeast_recipes\31600-pdf.pdf.crypt


In total 2847 files where encrypted

In [4]:
len(df)

2847

From the below stream:smb log we ca see that changes are done by 10.0.4.4 

In [30]:
df=SPL("""index=botsv2 sourcetype="stream:smb" filename=*.crypt | sort _time | table _time,src_ip,dest_ip,filename""")
df[:10]

,_time,src_ip,dest_ip,filename
0,2017-08-18T21:52:17.989+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
1,2017-08-18T21:52:18.010+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
2,2017-08-18T21:52:18.145+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
3,2017-08-18T21:52:18.174+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
4,2017-08-18T21:52:18.244+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
5,2017-08-18T21:52:18.884+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
6,2017-08-18T21:52:18.908+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
7,2017-08-18T21:52:19.192+00:00,10.0.4.4,10.0.1.101,.DS_Store.crypt
8,2017-08-18T21:52:19.218+00:00,10.0.4.4,10.0.1.101,Home.crypt
9,2017-08-18T21:52:19.246+00:00,10.0.4.4,10.0.1.101,Home.crypt


Searching for 10.0.4.4 we can find a windows eventlog where we can see that the computers hostname is MACLORY-AIR13S and the user is mallory.kraeusen

In [42]:
df=SPL("""index=botsv2 src_ip=10.0.4.4 sourcetype="wineventlog" """)

In [48]:
print(df["_time"][0])
print(df["Message"][0])

2017-08-18T21:27:35.000+00:00
An account was successfully logged on.

Subject:
	Security ID:		NULL SID
	Account Name:		-
	Account Domain:		-
	Logon ID:		0x0

Logon Type:			3

Impersonation Level:		Impersonation

New Logon:
	Security ID:		FROTHLY\mallory.kraeusen
	Account Name:		mallory.kraeusen
	Account Domain:		FROTHLY
	Logon ID:		0x33793D
	Logon GUID:		{00000000-0000-0000-0000-000000000000}

Process Information:
	Process ID:		0x0
	Process Name:		-

Network Information:
	Workstation Name:	MACLORY-AIR13S
	Source Network Address:	10.0.4.4
	Source Port:		49735

Detailed Authentication Information:
	Logon Process:		NtLmSsp 
	Authentication Package:	NTLM
	Transited Services:	-
	Package Name (NTLM only):	NTLM V2
	Key Length:		128

This event is generated when a logon session is created. It is generated on the computer that was accessed.

The subject fields indicate the account on the local system which requested the logon. This is most commonly a service such as the Server service, or a loc

Looking at the local logs on MACLORY-ARI13 in the query below it looks like the malware started encrypting the local files about 1-2min before getting to the network drive

In [36]:
SPL(""" index=botsv2 host=MACLORY-AIR13 *.crypt | sort _time | head 10 | table _time, sourcetype, columns.action, columns.target_path, process""")

,_time,sourcetype,columns.action,columns.target_path,process
0,2017-08-18T21:50:40.060+00:00,osquery_info,NaN,NaN,NaN
1,2017-08-18T21:50:40.060+00:00,osquery_warning,NaN,NaN,NaN
2,2017-08-18T21:50:43.000+00:00,osquery_results,ATTRIBUTES_MODIFIED,/Users/mallorykraeusen/Downloads/Office2016_Patcher_For_OSX.torrent.crypt,NaN
3,2017-08-18T21:50:43.000+00:00,osquery_results,MOVED_TO,/Users/mallorykraeusen/Downloads/Office2016_Patcher_For_OSX.torrent.crypt,NaN
4,2017-08-18T21:50:43.000+00:00,osquery_results,ATTRIBUTES_MODIFIED,/Users/mallorykraeusen/Downloads/Office2016_Patcher_For_OSX.torrent.crypt,NaN
5,2017-08-18T21:50:43.000+00:00,osquery_results,DELETED,/Users/mallorykraeusen/Downloads/Office2016_Patcher_For_OSX.torrent.crypt,NaN
6,2017-08-18T21:50:43.000+00:00,osquery_results,CREATED,/Users/mallorykraeusen/Downloads/Office2016_Patcher_For_OSX.torrent.crypt,NaN
7,2017-08-18T21:50:43.000+00:00,osquery_results,ATTRIBUTES_MODIFIED,/Users/mallorykraeusen/Downloads/Office 2016 Patcher.app.crypt,NaN
8,2017-08-18T21:50:43.000+00:00,osquery_results,MOVED_TO,/Users/mallorykraeusen/Downloads/Office 2016 Patcher.app.crypt,NaN
9,2017-08-18T21:50:43.000+00:00,osquery_results,DELETED,/Users/mallorykraeusen/Downloads/Office 2016 Patcher.app.crypt,NaN


In [39]:
#TODO find inital exploit